In [1]:
import streamlit as st
import matplotlib.pyplot as plt
import numpy as np
from monai.transforms import Compose, Invertd, SaveImaged
from src.utils import Config, ImageProcessor
from src.data_preparation import DataHandling, LoaderFactory
from src.vis import visualize_axial_slice, visualize_coronal_slice, dash_plot_artifact, display_patient_coronal
from src.model_manager import ModelLoader
import pandas as pd
import os

config_file = 'src/config.json'
config = Config(config_file)
data_handler = DataHandling(
    config.artifact_dir,
    train_mode=config.train_mode,
    target_mode=config.target_mode,
    external_centers=config.external_centers,
    train_percent=config.train_percent,
    val_percent=config.val_percent,
    test_percent=config.test_percent
    )


train_files = data_handler.get_data_split('train')
val_files = data_handler.get_data_split('val')
test_files = data_handler.get_data_split('test')


loader_factory = LoaderFactory(
    train_files=train_files,
    val_files=val_files,
    test_files=test_files,
    patch_size=config.patch_size,
    spacing=config.spacing,
    spatial_size=config.spatial_size,
    normalize=config.normalize
    )


# Get the DataLoader for each dataset type
test_loader = loader_factory.get_loader('test', batch_size=config.batch_size['test'], num_workers=config.num_workers['test'], shuffle=False)
test_name = [(os.path.splitext(os.path.basename(file_info['image']))[0], os.path.splitext(os.path.basename(file_info['target']))[0]) for file_info in test_files]
image_files = [file_info['image'] for file_info in test_files]
target_files = [file_info['target'] for file_info in test_files]

# Create a list of dictionaries containing the patient data
patient_data = [{
    'name': os.path.splitext(name_pair[0])[0],
    'image': image_path,
    'target': target_path
} for name_pair, image_path, target_path in zip(test_name, image_files, target_files)]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(patient_data)
hint = 'dl4_23'  # Replace with the actual hint
n = 85  # You can choose any valid slice index

# Create an instance of ImageProcessor
processor = ImageProcessor(config.artifact_output_dir, config.nac_factor_ga, config.mac_factor_ga, hint)

# Assuming df_patients and test_files are already defined
df = processor.load_and_store_images_to_df(df, test_files)

Requested device: cuda:0
CUDA is not available. Switching to CPU.
No data was picked from the external centers.
Not enough data to split for center C2. Minimum required is 10, but found 4.
Number of training files: 38
Number of validation files: 9
Number of test files: 9


In [3]:
# Assuming df is your DataFrame
def save_df_to_pickle(df, filename='Artifact_data.pkl'):
    """Save the DataFrame to a Pickle file."""
    df.to_pickle(filename)
    print(f"DataFrame saved to {filename}")

# Example usage
# df = your_dataframe_here
save_df_to_pickle(df)

DataFrame saved to Artifact_data.pkl


In [5]:
def load_df_from_pickle(filename='/students/2023-2024/master/Shahpouri/DATA/Artifact_data.pkl'):
    """Load the DataFrame from a Pickle file."""
    try:
        df = pd.read_pickle(filename)
        print(f"DataFrame loaded from {filename}")
        return df
    except FileNotFoundError:
        print(f"File {filename} not found.")
        return None

# Example usage
df = load_df_from_pickle()

df.head()

DataFrame loaded from Artifact_data.pkl


,name,image,target,dl_image_path,image_matrix,target_matrix,dl_image_matrix,difference_matrices
0,001468_C1_RZ_,/students/2023-2024/master/Shahpouri/DATA/ARTI...,/students/2023-2024/master/Shahpouri/DATA/ARTI...,/students/2023-2024/master/Shahpouri/ARTIFACT_...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.01271765329875052, 0.009126178920269012, ...","[[[-0.002543530659750104, -0.00182523578405380..."
1,001695_C1_RZ_,/students/2023-2024/master/Shahpouri/DATA/ARTI...,/students/2023-2024/master/Shahpouri/DATA/ARTI...,/students/2023-2024/master/Shahpouri/ARTIFACT_...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.003006885526701808, 0.0006544784992001951...","[[[-0.0006013771053403616, -0.0001308956998400..."
2,003242_C3_KH_,/students/2023-2024/master/Shahpouri/DATA/ARTI...,/students/2023-2024/master/Shahpouri/DATA/ARTI...,/students/2023-2024/master/Shahpouri/ARTIFACT_...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,003257_C3_KH_,/students/2023-2024/master/Shahpouri/DATA/ARTI...,/students/2023-2024/master/Shahpouri/DATA/ARTI...,/students/2023-2024/master/Shahpouri/ARTIFACT_...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,004254_C4_SH_,/students/2023-2024/master/Shahpouri/DATA/ARTI...,/students/2023-2024/master/Shahpouri/DATA/ARTI...,/students/2023-2024/master/Shahpouri/ARTIFACT_...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.004280496796127409, 0.002915181976277381,...","[[[-0.0008560993592254817, -0.0005830363952554..."


In [2]:

hint = 'dl4_23'  # Replace with the actual hint

# Create an instance of ImageProcessor
processor = ImageProcessor(config.artifact_output_dir, config.nac_factor_ga, config.mac_factor_ga, hint)

# Assuming df_patients and test_files are already defined
df = processor.load_and_store_images_to_df(df, test_files)
df.head()



KeyboardInterrupt: 

In [ ]:
import numpy as np

# Function to display images for a sample patient
def plot_sample_patient(df, patient_name, n):
    # Find the patient data in the DataFrame
    patient_data = df[df['name'] == patient_name]
    
    if patient_data.empty:
        print(f"No data found for patient {patient_name}")
        return
    
    # Extract the image matrices
    image = patient_data['image_matrix'].values[0]
    target = patient_data['target_matrix'].values[0]
    dl_image = patient_data['dl_image_matrix'].values[0]
    difference_image = patient_data['difference_matrices'].values[0]
    


    # Display the images using the provided function
    display_patient_coronal(patient_name, image, target, dl_image, difference_image, n, cmp="gist_yarg")

# Choose a slice index for visualization
n = 85  # You can choose any valid slice index
sample_patient_name = '004254_C4_SH_'  # Replace with an actual patient name from your DataFrame
plot_sample_patient(df, sample_patient_name)


In [ ]:
import nibabel as nib
hint = 'dl4_23'
nac_factor=2
mac_factor=5
patient_names = [name_pair[0] for name_pair in test_name]

i = 1
for file_info, name in zip(test_files, patient_names):
    image_path = file_info['image']
    target_path = file_info['target']
 
    # Remove any potential file extensions from patient_folder_name_image
    patient_folder_name_image = os.path.splitext(name)[0]
    dl_image_path = find_dl_image_path(config.artifact_output_dir, patient_folder_name_image, hint)
    if dl_image_path is None:
        continue
    try:
        image = (nib.load(image_path).get_fdata()) * nac_factor
        target = (nib.load(target_path).get_fdata()) * mac_factor
        dl_image = (nib.load(dl_image_path).get_fdata()) * mac_factor
    except Exception as e:
        print(f"Error loading or processing images for {patient_folder_name_image}: {e}")
        continue

    difference_image = (target - dl_image) / mac_factor
    # print(np.max(difference_image), np.min(difference_image))
    difference_image = np.clip(difference_image, -1, 1)

    
    n = 85

    patient_name = patient_names[1]
    display_patient_coronal(patient_name, image, target, dl_image, difference_image, n, cmp ="gist_yarg")
    i +=1
    if i == 2:
        break

In [ ]:
test_files

In [ ]:
# from matplotlib.colors import LinearSegmentedColormap
# import matplotlib.pyplot as plt
# import numpy as np

# def display_patient_coronal(patient_folder_name, image, target, dl_image, difference_image, n, cmp="gray"):
#     """
#     Display medical images for a patient: input, target, deep learning output, and the difference.
#     """
#     colors = [(0.00, "red"), (0.40, "white"), (0.80, "white"), (1.00, "blue")]
#     cmap_name = 'custom_seismic_more_white'
#     cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=12)

#     fig, axs = plt.subplots(1, 4, figsize=(12, 6), gridspec_kw={'wspace': 0, 'hspace': 0})
#     cbar_ax = fig.add_axes([0.92, 0.1, 0.02, 0.8])

#     # Turn off axes
#     for ax in axs:
#         ax.axis('off')

#     # Configure vmin and vmax for each image type
#     nac_display_range = (np.percentile(image, 0), np.percentile(image, 99.9))
#     mac_display_range = (np.percentile(target, 0), np.percentile(target, 99.9))
#     dl_display_range = (np.percentile(dl_image, 0), np.percentile(dl_image, 99.9))

#     # Input Image
#     input_slice = np.rot90(image[:, n, :])
#     im = axs[0].imshow(input_slice, cmap=cmp, vmin=nac_display_range[0], vmax=nac_display_range[1])

#     # Target Image
#     axs[1].set_title(patient_folder_name)
#     target_slice = np.rot90(target[:, n, :])
#     axs[1].imshow(target_slice, cmap=cmp, vmin=mac_display_range[0], vmax=mac_display_range[1])

#     # DL Image
#     dl_slice = np.rot90(dl_image[:, n, :])
#     axs[2].imshow(dl_slice, cmap=cmp, vmin=dl_display_range[0], vmax=dl_display_range[1])

#     # Difference Image
#     difference_slice = np.rot90(difference_image[:, n, :])
#     im_diff = axs[3].imshow(difference_slice, cmap=cm, vmin=-1, vmax=1)

#     # Add colorbars
#     fig.colorbar(im, ax=axs[0], orientation='vertical')
#     fig.colorbar(im, ax=axs[1], orientation='vertical')
#     fig.colorbar(im, ax=axs[2], orientation='vertical')
#     fig.colorbar(im_diff, ax=axs[3], orientation='vertical')

#     plt.show()


# n = 50

# display_patient_coronal(patient_folder_name, image, target, dl_image, difference_image, n)


In [ ]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import numpy as np

def display_patient_coronal(patient_folder_name, image, target, dl_image, difference_image, n, cmp="gray"):
    """
    Display medical images for a patient: input, target, deep learning output, and the difference.
    """
    colors = [(0.00, "orange"), (0.40, "white"), (0.8, "white"), (1.00, "blue")]
    cmap_name = 'custom_seismic_more_white'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=12)

    fig, axs = plt.subplots(1, 4, figsize=(12, 4), gridspec_kw={'wspace':0.3, 'hspace':0})

    # Turn off axes
    for ax in axs:
        ax.axis('off')

    # Configure vmin and vmax for each image type
    nac_display_range = (np.percentile(image, 0), np.percentile(image, 99.9))
    mac_display_range = (np.percentile(target, 0), np.percentile(target, 99.9))
    dl_display_range = (np.percentile(dl_image, 0), np.percentile(dl_image, 99.9))

    # Input Image
    input_slice = np.rot90(image[:, n, :])
    im0 = axs[0].imshow(input_slice, cmap=cmp, vmin=nac_display_range[0], vmax=nac_display_range[1])
    fig.colorbar(im0, ax=axs[0], fraction=0.046, pad=0.04)

    # Target Image
    axs[1].set_title(patient_folder_name)
    target_slice = np.rot90(target[:, n, :])
    im1 = axs[1].imshow(target_slice, cmap=cmp, vmin=mac_display_range[0], vmax=mac_display_range[1])
    fig.colorbar(im1, ax=axs[1], fraction=0.046, pad=0.04)

    # DL Image
    dl_slice = np.rot90(dl_image[:, n, :])
    im2 = axs[2].imshow(dl_slice, cmap=cmp, vmin=dl_display_range[0], vmax=dl_display_range[1])
    fig.colorbar(im2, ax=axs[2], fraction=0.046, pad=0.04)

    # Difference Image
    difference_slice = np.rot90(difference_image[:, n, :])
    im3 = axs[3].imshow(difference_slice, cmap=cm, vmin=-1, vmax=1)
    fig.colorbar(im3, ax=axs[3], fraction=0.046, pad=0.04)

    plt.show()



In [ ]:
from utils import find_dl_image_path, normalize_data
# from vis import display_patient_coronal


hint = "dl_final_3_29"
hint = 'gamodel_3_18_onfdg'
hint = 'dl4_23'
hint = 'adcm_4_26'
hint = "final_4_26"
hint = 'comb_4_5_onfdg'
nac_factor=3
mac_factor=7

i = 1
for file_info, name_pair in zip(test_files, test_name):
    image_path = file_info['image']
    target_path = file_info['target']
    patient_folder_name_image, _ = name_pair

    dl_image_path = find_dl_image_path(fdg_output_dir, patient_folder_name_image, hint)
    if dl_image_path is None:
        print(f"No DL image found for {patient_folder_name_image}")
        continue

    # image = normalize_data(nib.load(image_path).get_fdata())
    # target = normalize_data(nib.load(target_path).get_fdata())
    # dl_image = normalize_data(nib.load(dl_image_path).get_fdata())

    image = (nib.load(image_path).get_fdata()) * nac_factor
    target = (nib.load(target_path).get_fdata()) * mac_factor
    dl_image = (nib.load(dl_image_path).get_fdata()) * mac_factor
    difference_image = (target - dl_image) 
    # print(np.max(difference_image), np.min(difference_image))
    difference_image = np.clip(difference_image, -1, 1)


    n = 90
    # n = 85
    # n = 90
    # n = 108
    # n = 100
    # n = 115
    
    display_patient_coronal(patient_folder_name_image, image, target, dl_image, difference_image, n, cmp ="jet")
    # i +=1
    # if i == 20:
    #     break


In [ ]:
# import nibabel as nib
# import matplotlib.pyplot as plt
# import numpy as np
# import os
# import glob
# from matplotlib.colors import LinearSegmentedColormap
# import numpy as np
# import matplotlib.pyplot as plt

# def apply_window(image, window_level, window_width):
#     """
#     Apply window level and width to an image.

#     Parameters:
#     - image: The input image as a NumPy array.
#     - window_level (WL): The center of the range of interest.
#     - window_width (WW): The width of the range of interest.

#     Returns:
#     - The windowed image.
#     """
#     lower_bound = window_level - window_width / 2
#     upper_bound = window_level + window_width / 2
#     windowed_image = np.clip(image, lower_bound, upper_bound)
#     windowed_image = (windowed_image - lower_bound) / window_width
#     return windowed_image

# window_level = 0  # Example values
# window_width = 1  # Example values

# hint = "dl3_18"

# # Function to find the DL image path for a given patient folder name
# def find_dl_image_path(patient_folder_name):
#     # Construct a glob pattern to search for DL images with the matching patient folder name
#     search_pattern = os.path.join(artifact_output, "**", f"{patient_folder_name}*{hint}.nii.gz")
#     found_paths = glob.glob(search_pattern, recursive=True)
#     if found_paths:
#         return found_paths[0]  # Return the first match
#     else:
#         return None  # No match found

# def normalize_data(data):
#     return (data - np.min(data)) / (np.max(data) - np.min(data))
# cmap = LinearSegmentedColormap.from_list(
#     'bwr', 
#     # ['purple', 'purple','darkviolet','white','white', 'lime', 'limegreen','limegreen']
#     ['purple','white','limegreen']
#     # ['purple', 'purple','purple','white','white', 'limegreen', 'limegreen','limegreen']
#     #['red', 'red','orangered','white','white', 'royalblue', 'blue','blue']
#     #['red', 'red','red','white','white', 'blue', 'blue','blue']
#     # ['red', 'red','red','tomato','white','white' , 'dodgerblue', 'blue', 'blue', 'blue']
# )
# # Loop over each file pair, find the matching DL image, and plot them
# for file_info, name_pair in zip(test_files, test_name):
#     image_path = file_info['image']
#     target_path = file_info['target']
#     patient_folder_name_image, _ = name_pair

#     dl_image_path = find_dl_image_path(patient_folder_name_image)
#     if dl_image_path is None:
#         print(f"No DL image found for {patient_folder_name_image}")
#         continue

#     # Load and normalize images
#     image = normalize_data(nib.load(image_path).get_fdata())
#     target = normalize_data(nib.load(target_path).get_fdata())
#     dl_image = normalize_data(nib.load(dl_image_path).get_fdata())
#     windowed_image = apply_window(image, window_level, window_width)
#     # Calculate the difference image and normalize to range -1 to 1
#     difference_image = target - dl_image
#     # difference_image = np.clip(difference_image, -1, 1)  # Ensure the difference is within -1 to 1

#     # difference_image = 2 * (difference_image - np.min(difference_image)) / (np.max(difference_image) - np.min(difference_image)) - 1

#     # image = nib.load(image_path).get_fdata()
#     # target = nib.load(target_path).get_fdata()
#     # dl_image = nib.load(dl_image_path).get_fdata()
#     # difference_image = target - dl_image
#     # # difference_image = np.clip(difference_image, -1, 1)  # Ensure the difference is within -1 to 1
#     n = 94

#     fig, axs = plt.subplots(1, 4, figsize=(20, 6))

#     plt.subplot(1, 4, 1)
#     plt.title(f"Input: {patient_folder_name_image}")
#     input_slice = np.rot90(image[:, n, :])
#     plt.imshow(input_slice, cmap="gist_yarg", vmin=0, vmax=0.1)
#     plt.axis('off')

#     plt.subplot(1, 4, 2)
#     plt.title("Target")
#     input_slice = np.rot90(target[:, n, :])
#     plt.imshow(input_slice, cmap="gist_yarg", vmin=0, vmax=0.1)
#     plt.axis('off')

#     plt.subplot(1, 4, 3)
#     plt.title("DL Image")
#     input_slice = np.rot90(dl_image[:, n, :])
#     plt.imshow(input_slice, cmap="gist_yarg", vmin=0, vmax=0.1)
#     plt.axis('off')

#     ax = plt.subplot(1, 4, 4)
#     plt.subplot(1, 4, 4)
#     plt.title("Difference")
#     # Using a diverging colormap to highlight differences
#     difference_display = plt.imshow(np.rot90(difference_image[:, n, :]), cmap=cmap, vmin=-0.1, vmax=0.1) # vmin=-np.max(np.abs(difference_image)), vmax=np.max(np.abs(difference_image)))
#     plt.axis('off')
#     fig.colorbar(difference_display, ax=axs[3])

#     plt.tight_layout()
#     plt.show()
#     # break
